In [1]:
import logging
import datetime

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import config
import modules.dataloaders as dataloaders

#import modules.model_CNV as cnv_model
# import modules.models.model_CNV_w1a1 as cnv_model
# import modules.models.model_CNV_w1a1_BIG as cnv_model
# import modules.models.model_CNV_w1a1_WIDE as cnv_model
# import modules.models.model_CNV_w1a1_linear4 as cnv_model
import modules.models.model_CNV_w1a1_DEEP_Resnet as cnv_model


#import modules.models.model_CNV_imagenet as cnv_model
# import modules.models.model_CNV_AIMET_imagenet as cnv_model
# import modules.models.model_CNV_AIMET_NoPadding_imagenet as cnv_model
# import modules.models.model_CNV_AIMET_NoPadding_FPGA_imagenet as cnv_model

import modules.loss as loss
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch
import modules.utils as utils

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [2]:
# from brevitas.export import export_onnx_qcdq
# from brevitas.export import export_qonnx

In [68]:
import importlib

<module 'modules.train_epoch' from '/home/gmoreno/uav/code/classifier_brevitas_2_finn/modules/train_epoch.py'>

In [124]:
importlib.reload(config)

<module 'config' from '/home/gmoreno/uav/code/classifier_brevitas_2_finn/config.py'>

In [133]:
importlib.reload(cnv_model)

<module 'modules.models.model_CNV_w1a1_DEEP_Resnet' from '/home/gmoreno/uav/code/classifier_brevitas_2_finn/modules/models/model_CNV_w1a1_DEEP_Resnet.py'>

In [126]:
importlib.reload(train_epoch)

<module 'modules.train_epoch' from '/home/gmoreno/uav/code/classifier_brevitas_2_finn/modules/train_epoch.py'>

# Logger

In [127]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('BED Classifier.\n' +  
            '\tOne Head.\n' +
            '\tWeighted for Precision.\n' +
            '\tBrevitas Default.\n'+ 
            '\tDataset images divided by 255.\n')

# Hyperparameters Log

In [128]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nBrevitas Config:')
print(f'\tFixed Point: {config.FIXED_POINT}')
print(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
print(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
print(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
print(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nBrevitas Config:')
logger.info(f'\tFixed Point: {config.FIXED_POINT}')
logger.info(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
logger.info(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')


Datasets Length
	Train and Val: Full

Load Model: False
Device: cuda
Optimizer:
	Learning Rate: 0.001
	Weight Decay: 0.001
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 2
	Scheduler threshold: 0.001
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 150

IMG DIMS:
	Width: 224
	Height: 224

Brevitas Config:
	Fixed Point: True
	Weights Bit Width: 4
	Big Layers Weights Bit Width: 4
	Bias Bit Width: 4
	Activations Bit Width: 4


# Dataloaders

In [6]:
train_loader = dataloaders.get_train_loader()
val_loader = dataloaders.get_val_loader()


TRAIN DFIRE dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


DFire Removed wrong images: 0
DFire empty images: 7833
DFire only smoke images: 4681
DFire only fire images: 944
DFire smoke and fire images: 3763

Train DFire dataset len: 17221

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 5994
FASDD only smoke images: 2541
FASDD only fire images: 105
FASDD smoke and fire images: 3911

Train FASDD UAV dataset len: 12551

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 3995
FASDD only smoke images: 1693
FASDD only fire images: 70
FASDD smoke and fire images: 2607

Val FASDD UAV dataset len: 8365

TRAIN FASDD CV dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


FASDD Removed wrong images: 0
FASDD empty images: 19600
FASDD only smoke images: 11708
FASDD only fire images: 6276
FASDD smoke and fire images: 10076

Train FASDD CV dataset len: 47660

Val FASDD CV dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


FASDD Removed wrong images: 0
FASDD empty images: 13066
FASDD only smoke images: 7804
FASDD only fire images: 4183
FASDD smoke and fire images: 6717

Val FASDD CV dataset len: 31770

Concatenate Train DFire and Train FASDD UAV datasets
Train dataset len: 29772
Concatenate with Val FASDD UAV dataset
Train dataset len: 38137
Concatenate with Train FASDD CV dataset
Train dataset len: 85797
Concatenate with Val FASDD CV dataset
Train dataset len: 117567

TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 1997
FASDD only smoke images: 846
FASDD only fire images: 35
FASDD smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 6533
FASDD only smoke images: 3902
FASDD only fire images: 2091
FASDD smoke and 

### Plot Some Train Pictures

In [129]:
for i, (img, label) in enumerate(train_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
    #plt.show()
    plt.close()
    break

### Plot Some Val Pictures

In [130]:
for i, (img, label) in enumerate(val_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
    plt.close()
    break

# Load Model

In [131]:
# import modules.models.model_CNV_w1a1_WIDE as cnv_model_big

In [134]:
# model = cnv_model.CNV(
#     num_classes = 2, 
#     weight_bit_width=4, 
#     act_bit_width=4, 
#     in_bit_width=8, 
# )  

# Models with Brevitas Tutorials imagenet style
# model = cnv_model.CNV().to(config.DEVICE)
# model = cnv_model.CNV_w1a1().to(config.DEVICE)
# model = cnv_model.CNV_w1a1_BIG().to(config.DEVICE)
# model = cnv_model_big.CNV_w1a1_WIDE().to(config.DEVICE)
# model = cnv_model.CNV_w1a1_linear4().to(config.DEVICE)
model = cnv_model.CNV_w1a1_DEEP_Resnet().to(config.DEVICE)

# model = cnv_model.CNV_AIMET().to(config.DEVICE)

### Optimizer and Scheduler

In [135]:
optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)



### Parameters

In [136]:
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 302948
Total parameters = 302948



### Check Model Shape: Random Input

In [137]:
dummy_input = np.random.rand(4, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32, device=config.DEVICE)
out_test = model(dummy_input)
print(f'Model shape is {out_test}')
#print(f'BED Model Arquitecture\n{cnv_model}')

Model shape is tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0', grad_fn=<AddBackward0>)


### Torchinfo

In [138]:
print(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                                                 Output Shape              Param #
CNV_w1a1_DEEP_Resnet                                                   [1, 2]                    --
├─ModuleList: 1-9                                                      --                        (recursive)
│    └─QuantIdentity: 2-1                                              [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-1                             [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-2                             [1, 3, 224, 224]          --
├─ModuleList: 1-64                                                     --                        (recursive)
│    └─QuantLinear: 2-140                                              --                        (recursive)
│    │    └─WeightQuantProxyFromInjector: 3-176                        --                        (recursive)
├─ModuleList: 1-9                                          

# Loss Function

In [139]:
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    logger.info(f'\nLoss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    logger.info(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Loggers and Plotters for Losses and Metrics

In [140]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Main Function to Train

In [141]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_smoke_precision = 0. #torch.tensor([0.])
    smoke_f1_min_save = 0.9 #torch.tensor([0.9])
    best_mean_f1 = 0.

    if start_epoch == 0:
        epochs_plot = []
    else:
        epochs_plot = [i for i in range(start_epoch)]    

    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
                
        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            device=config.DEVICE)
        
        train_losses_logger.update_metrics(train_losses)
        train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN Stats", train_losses, train_metrics))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                device=config.DEVICE)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_metrics(val_losses)
            val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL Stats", val_losses, val_metrics))
            
        epochs_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_metrics(),
            val_losses_logger.get_metrics(),
            epochs_plot)

        metrics_plotter.plot_all_metrics(
            train_metrics_logger.get_metrics(),
            val_metrics_logger.get_metrics(),
            epochs_plot)

        lr_logger.plot_lr(epochs_plot)
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.4f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.4f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  #+ '.onnx'
            utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # # Save model if precision increases and F1 > 0.9
        # if ( best_smoke_precision < val_metrics['Precision'][0] ) and ( val_metrics['F1'][0] > smoke_f1_min_save ) :
        #     best_smoke_precision = val_metrics['Precision'][0]
        #     print(f"\nSaving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     logger.info(f"Saving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     save_precision_name = f'best_smoke__precision={np.round(best_smoke_precision, decimals=4)}__epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_precision_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
        #     save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_precision_name #+ '.onnx'
        #     utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
            
        # Save model if precision > 0.9 and recall > 0.9
        if ( val_metrics['Precision'][0] > 0.9 ) and ( val_metrics['Recall'][0] > 0.9 ) :
            print("\nSaving model with precision > 0.9 and recall > 0.9")
            logger.info("Saving model with precision > 0.9 and recall > 0.9")
            save_pre_name = f'smoke__precision={np.round(val_metrics["Precision"][0], decimals=4)}__' 
            save_rec_name = f'recall={np.round(val_metrics["Recall"][0], decimals=4)}__'
            save_pre_rec_name = save_pre_name + save_rec_name + f'epoch={epoch}'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name #+ '.onnx'
            utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # Save model if best mean F1 increases
        val_f1_mean = (val_metrics['F1'][0] + val_metrics['F1'][1]) / 2
        if (val_f1_mean > best_mean_f1) :
            best_mean_f1 = val_f1_mean
            print(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            logger.info(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            save_f1_name = 'best_mean_F1'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name #+ '.onnx'
            utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_classifier.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

In [142]:
# print(len(val_losses_logger.total))

# Training Loop

In [143]:
if __name__ == "__main__":
    
    print("Starting script\n")
    logger.info("Starting script\n")
    
    model = train_loop(model)

Starting script


***Start Training: 02:03:01


=== EPOCH 0/149 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1836/1836 [04:58<00:00,  6.15it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
71.129      |37.125      |34.004      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:36<00:00, 10.27it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
61.224      |32.081      |29.143      
SMOKE -> Precision: 0.7391 - Recall: 0.5332 - Accuracy: 0.6912 - F1: 0.6195
FIRE -> Precision: 0.6984 - Recall: 0.5485 - Accuracy: 0.7769 - F1: 0.6144

Saving model with new best validation loss: 61.2242
Model exported to ONNX: experiments_resnet/test_v03__w1a1__DEEP_Resnet__clip_convs_Linear4ReLU__400k_full_ds/onnx/BNN_BED_classifier__best_loss
Saving model with best Mean F1: 0.6169
Model exported to ONNX: experiments_resnet/test_v03__w1a1__DEEP_Resnet__clip_convs_Linear4ReLU__400k_full_ds/onnx/BNN_BED_classifier__best_mean_F1

=== EPOCH 1/149 ===
Learning Rate = 0.001



...

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
27.420      |17.786      |9.634       
SMOKE -> Precision: 0.8989 - Recall: 0.7818 - Accuracy: 0.8557 - F1: 0.8363
FIRE -> Precision: 0.9108 - Recall: 0.9143 - Accuracy: 0.9432 - F1: 0.9125

=== EPOCH 149/149 ===
Learning Rate = 1e-06



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1836/1836 [05:05<00:00,  6.01it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
29.538      |18.281      |11.258      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:36<00:00, 10.38it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
29.513      |19.163      |10.349      
SMOKE -> Precision: 0.8533 - Recall: 0.8156 - Accuracy: 0.8470 - F1: 0.8340
FIRE -> Precision: 0.8882 - Recall: 0.9343 - Accuracy: 0.9406 - F1: 0.9107

***Script finished: 16:21:20

Time elapsed: 14:18:19.094563


### Train MORE

In [144]:
# if __name__ == "__main__":
    
#     print("Train More script\n")
#     logger.info("Train More\n")
    
#     model = train_loop(model, start_epoch=100, epochs_to_train=50)

# Test with DFire MINI Dataset: Train and Test

In [145]:
# import importlib
# importlib.reload(config)
# importlib.reload(dataloaders)

In [101]:
train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TRAIN DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 20
DFire only smoke images: 45
DFire only fire images: 5
DFire smoke and fire images: 30

Test dataset len: 100

TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Load Checkpoint with Best F1 Mean

In [146]:
model_path = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__best_mean_F1.pt'

In [147]:
utils.load_checkpoint(
    model_path, 
    model=model, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    device=config.DEVICE)

Loading Model. Trained during 130 epochs


130

### Whole Test Loader, to check it is the same as training

In [148]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:37<00:00, 10.25it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
26.795      |17.741      |9.054       
SMOKE -> Precision: 0.8779 - Recall: 0.8116 - Accuracy: 0.8580 - F1: 0.8435
FIRE -> Precision: 0.9016 - Recall: 0.9390 - Accuracy: 0.9470 - F1: 0.9199


In [149]:
logger.info('\nTesting with FULL TEST LOADER')  
#logger.info(val_losses)
logger.info(val_metrics)

### Train DFire MINI

In [150]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=train_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 48.95it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
0.674       |0.423       |0.251       
SMOKE -> Precision: 0.9200 - Recall: 0.6133 - Accuracy: 0.6700 - F1: 0.7360
FIRE -> Precision: 0.8824 - Recall: 0.8571 - Accuracy: 0.9100 - F1: 0.8696


In [151]:
logger.info('\nTesting with DFire MINI TRAIN after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

### Test DFire MINI

In [152]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 37.03it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
0.600       |0.377       |0.223       
SMOKE -> Precision: 0.8947 - Recall: 0.7727 - Accuracy: 0.7667 - F1: 0.8293
FIRE -> Precision: 0.7857 - Recall: 1.0000 - Accuracy: 0.9000 - F1: 0.8800


In [153]:
logger.info('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

# Convert the Model to BIPOLAR OUT

In [154]:
import brevitas.nn as qnn
import torch.nn as nn

In [155]:
class CNV_BIPOLAR_OUT(nn.Module):
    def __init__(self, base_model):
        super(CNV_BIPOLAR_OUT, self).__init__()
        self.base_model = base_model
        self.qnt_output = qnn.QuantIdentity(
            quant_type='binary', 
            scaling_impl_type='const',
            bit_width=1, min_val=-1.0, max_val=1.0)

    def forward(self, x):
        x = self.base_model(x)
        x = self.qnt_output(x)
        return x

In [156]:
cnv_bipolar_out = CNV_BIPOLAR_OUT(model).to(config.DEVICE)

### New Evaluation for BIPOLAR Out Model

In [157]:
from tqdm import tqdm

def eval_bipolar_fn(loader, model, device):
    
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        yhat = model(x)

        # print(y.shape)
        # print(yhat.shape)
        
        yhat[yhat < 1] = 0
    
        metrics.precision_metric.update(yhat, y)
        metrics.recall_metric.update(yhat, y)
        metrics.accuracy_metric.update(yhat, y)
        metrics.f1_metric.update(yhat, y)
   
    precision = metrics.precision_metric.compute()
    recall = metrics.recall_metric.compute()
    accuracy = metrics.accuracy_metric.compute()
    f1 = metrics.f1_metric.compute()
    
    metrics.precision_metric.reset()
    metrics.recall_metric.reset()
    metrics.accuracy_metric.reset()
    metrics.f1_metric.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

### Full DS

In [158]:
cnv_bipolar_out.eval()
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=val_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:36<00:00, 10.43it/s]

SMOKE -> Precision: 0.8779 - Recall: 0.8116 - Accuracy: 0.8580 - F1: 0.8435
FIRE -> Precision: 0.9016 - Recall: 0.9390 - Accuracy: 0.9470 - F1: 0.9199


### Mini Train

In [159]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=train_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 82.77it/s]

SMOKE -> Precision: 0.9200 - Recall: 0.6133 - Accuracy: 0.6700 - F1: 0.7360
FIRE -> Precision: 0.8824 - Recall: 0.8571 - Accuracy: 0.9100 - F1: 0.8696


### Mini Test

In [160]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=test_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 60.08it/s]

SMOKE -> Precision: 0.8947 - Recall: 0.7727 - Accuracy: 0.7667 - F1: 0.8293
FIRE -> Precision: 0.7857 - Recall: 1.0000 - Accuracy: 0.9000 - F1: 0.8800


# Export Bipolar to QONNX

In [161]:
save_f1_name = 'best_mean_F1'
save_bipolar_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '__BIPOLAR_Out'
utils.export_onnx(cnv_bipolar_out, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_bipolar_onnx, config.DEVICE)

Model exported to ONNX: experiments_resnet/test_v03__w1a1__DEEP_Resnet__clip_convs_Linear4ReLU__400k_full_ds/onnx/BNN_BED_classifier__best_mean_F1__BIPOLAR_Out


# Quant INT Out

In [118]:
# from brevitas.core.quant import QuantType
# from modules.common_imagenet import CommonIntActQuant
# from brevitas.core.restrict_val import RestrictValueType

In [119]:
# class CNV_QINT_OUT(nn.Module):
#     def __init__(self, base_model):
#         super(CNV_QINT_OUT, self).__init__()
#         self.base_model = base_model
#         self.qnt_output = qnn.QuantIdentity( 
#             act_quant = CommonIntActQuant,
#             bit_width = 8,
#             min_val = -1.0,
#             max_val = 1.0 - 2.0 ** (-7),
#             narrow_range = False, 
#             restrict_scaling_type = RestrictValueType.POWER_OF_TWO)

#     def forward(self, x):
#         x = self.base_model(x)
#         x = self.qnt_output(x)
#         return x

In [120]:
# cnv_qint_out = CNV_QINT_OUT(model).to(config.DEVICE)

In [121]:
# save_f1_name = 'best_mean_F1'
# save_qint_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '__QINT_Out'
# utils.export_onnx(cnv_qint_out, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_qint_onnx, config.DEVICE)

In [122]:
# for param in model.parameters():
#     print(param)